In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Read train data from csv file

In [ ]:
data=pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')

In [ ]:
data.head()

In [ ]:
data.info()

We have Gender,Vehicle_Age,Vehicle_Damage as object data type. Lets convert them into numeric type.

In [ ]:
data.Gender.unique()

In [ ]:
gender_map={'Male':1,'Female':2}
data['Gender']=data['Gender'].map(gender_map)

In [ ]:
data.Vehicle_Age.unique()

In [ ]:
vehicle_map={'> 2 Years':2,'1-2 Year':1,'< 1 Year':0}
data["Vehicle_Age"]=data["Vehicle_Age"].map(vehicle_map)

In [ ]:
data.Vehicle_Damage.unique()

In [ ]:
vehicle_damage_map={"Yes":1,"No":0}
data["Vehicle_Damage"]=data["Vehicle_Damage"].map(vehicle_damage_map)

Divide the train dataset into X and Y attributes

In [ ]:
X=data.iloc[:,:-1]
Y=data['Response']

In [ ]:
X.head()

In [ ]:
Y.head()

Feature Selection

In [ ]:
data.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr=data.iloc[:,:-1].corr()
top_features=corr.index
plt.figure(figsize=(20,20))
sns.heatmap(data[top_features].corr(),annot=True)
plt.show()

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info=mutual_info_classif(X,Y)

In [ ]:
mutual_data=pd.Series(mutual_info,index=X.columns)
mutual_data.sort_values(ascending=False)

Removing the last 4 features as they are contributing to very little here.

In [ ]:
X=data.drop(['Region_Code','Annual_Premium','id','Vintage'],axis=1)

Selecting the model :- XGBClassifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X.values,Y.values,test_size=0.25,random_state=50)

In [ ]:
from xgboost.sklearn import XGBClassifier
model=XGBClassifier(eta=0.1,max_Depth=6,min_child_weight=1,n_estimators=100)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
print("Accuracy")
print( round(metrics.accuracy_score(y_test, model.predict(X_test))*100,2),'%')